In [1]:
from landlab import RasterModelGrid
from landlab.plot.imshow import imshow_node_grid
from landlab.components import PresFlowNetwork, MeltCreep
import numpy as np

In [2]:
pwd()

u'/home/mcoving/github/landlab/landlab/components/conduit_networks/example'

In [37]:
mg = RasterModelGrid((40,50),100)
junc_elev = mg.add_zeros('node', 'junction__elevation')
#d_h = mg.add_ones('link', 'hydraulic__diameter')
d_h = np.random.rand(mg.number_of_links)
mg.add_field('link','hydraulic__diameter', d_h)
h = mg.add_zeros('node', 'hydraulic__head')
thickness = 500.*np.ones(mg.number_of_nodes)
Z = mg.add_field('node', 'ice__thickness', thickness)
#a = mg.add_ones('link', 'head_loss__exponent')
#f = mg.add_ones('link', 'friction__factor')
#h = np.random.rand(mg.number_of_nodes)
Q = mg.add_zeros('link', 'conduit__discharge')
#net_node_flux = mg.add_ones('node', 'net_node_flux', noclobber=False)
#set boundary node head

h[mg.nodes_at_left_edge] = 400.
h[mg.nodes_at_right_edge] = 300.
h[mg.nodes_at_top_edge] = 0.
h[mg.nodes_at_bottom_edge] = 0.
mg.set_closed_boundaries_at_grid_edges(False,True,False,True)
Q[mg.active_links] = 1.
n_core = mg.number_of_core_nodes
links = mg.links_at_node
print "Number of links = ", mg.number_of_links
print "Number of nodes = ", mg.number_of_nodes
print "Number of active links = ", mg.number_of_active_links
print "Number of core nodes = ", mg.number_of_core_nodes


Number of links =  3910
Number of nodes =  2000
Number of active links =  3638
Number of core nodes =  1824


In [38]:

pfn = PresFlowNetwork(mg)


In [39]:
pfn.run_one_step()

Number of iterations = 1 tolerance = 1.28607564541
Number of iterations = 2 tolerance = 0.898483034208
Number of iterations = 3 tolerance = 0.674241010494
Number of iterations = 4 tolerance = 0.4240247216
Number of iterations = 5 tolerance = 0.23943412438
Number of iterations = 6 tolerance = 0.137171233481
Number of iterations = 7 tolerance = 0.0776048861004
Number of iterations = 8 tolerance = 0.0441898412133
Number of iterations = 9 tolerance = 0.0216498570859
Number of iterations = 10 tolerance = 0.0108681837015
Number of iterations = 11 tolerance = 0.006099634995
Number of iterations = 12 tolerance = 0.00298883633737
Number of iterations = 13 tolerance = 0.00127482075344
Number of iterations = 14 tolerance = 0.000536690929679


In [11]:
%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [12]:
figure()
imshow_node_grid(mg, h)

In [50]:
mc = MeltCreep(mg, dt=100000.)

In [59]:
figure()
pfn.run_one_step()
mc.run_one_step()
imshow_node_grid(mg, h)

Number of iterations = 1 tolerance = 4.16270360809e-06


In [60]:
mean(d_h)#doesn't seem to be updating

0.50219688715632504

In [47]:
Q

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])